In [1]:
!pip install transformers

In [2]:
import numpy as np
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [3]:
import nltk
#nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
#nltk.download('all')

In [4]:
from transformers import AutoTokenizer, TFAutoModel

/home/apasalic/anaconda/envs/nlp_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
#FinanceInc/finbert-pretrain
finbert_tokenizer = AutoTokenizer.from_pretrained("FinanceInc/finbert-pretrain", from_pt=True)
finbert_model = TFAutoModel.from_pretrained("FinanceInc/finbert-pretrain", from_pt=True)

2024-01-27 20:54:06.096251: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-27 20:54:07.602372: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trai

In [6]:
finbert_model.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109751808 
                                                                 
Total params: 109,751,808
Trainable params: 109,751,808
Non-trainable params: 0
_________________________________________________________________


In [7]:
import json

with open('./data/gopikasr/DataChatBot.json') as f:
    data = json.load(f)

for i in data['intents'][:5]:
    print(i)

{'tag': 'welcome', 'patterns': ['Hi', 'Hello', 'How are you', 'whats up', 'How do you do', 'How’s the day?', 'Hey are you there', 'How is everything going', 'glad to meet you'], 'responses': ['Hello', 'Glad to see you again', 'Hi there, How can I help  you', 'Hello! what you are looking for?', 'Hello, how may I help you']}
{'tag': 'endingnote', 'patterns': ['see you. bye', 'Good Bye', 'ok then bye', 'That’ s enough for me', ' Im leaving', 'See u later! Goodbye', 'nice to meet you', 'The bargaining session was fun!I’m happy that I got it at cheap price, see u again', 'Enjoyed the experience, good bye'], 'responses': ['Hey, Thank you for visiting', 'Hope to see you again, it was nice talking with you', ' wish you a good day! Hope to have a talk with you later', 'Thanks for your time with us.Hope you enjoyed it and satisfied. See you later', 'Sad to see you go. Come whenever you need to bye from us. Stay intouch.']}
{'tag': 'name', 'patterns': ['whats you name?', 'who are you?', 'what can

In [8]:
def QnA_dict(data):
    QnA = {'questions':[], 'answers':[], 'tags':[]}
    for intent in data['intents']:
        for pattern in intent['patterns']:
            for response in intent['responses']:
                QnA['questions'].append(pattern)
                QnA['answers'].append(response)
                QnA['tags'].append(intent['tag'])
    return QnA 

In [9]:
qna = QnA_dict(data)

print(len(qna['questions']))

705


In [17]:
import pandas as pd
qna_df=pd.DataFrame(qna)
qna_df.to_csv('qna_df.csv',index=False)

In [18]:
qna_df

,questions,answers,tags
0,Hi,Hello,welcome
1,Hi,Glad to see you again,welcome
2,Hi,"Hi there, How can I help you",welcome
3,Hi,Hello! what you are looking for?,welcome
4,Hi,"Hello, how may I help you",welcome
...,...,...,...
700,Thank you,You are welcome,Thank you
701,Thank you,No problem. Its my pleasure to help you,Thank you
702,Thank you so much for your help,Its my pleasure.,Thank you
703,Thank you so much for your help,You are welcome,Thank you


In [ ]:
qna_df_reddit = pd.read_csv('qna_df_reddit.csv')

In [21]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Define the stop words
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove special characters
    text = re.sub(r'\W', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenization
    # tokens = nltk.word_tokenize(text)
    # Remove stop words and lemmatize the words
    # tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return text

input_text = '\n**User Report**| | | |\n:--|:--|:--|:--\n**Total Submissions**|10|**First Seen In WSB**|2 years ago\n**Total Comments**|332|**Previous Best DD**|\n**Account Age**|3 years|[^scan ^comment ](https://www.reddit.com/message/compose/?to=VisualMod&subject=scan_comment&message=Replace%20this%20text%20with%20a%20comment%20ID%20(which%20looks%20like%20h26cq3k\\)%20to%20have%20the%20bot%20scan%20your%20comment%20and%20correct%20your%20first%20seen%20date.)|[^scan ^submission ](https://www.reddit.com/message/compose/?to=VisualMod&subject=scan_submission&message=Replace%20this%20text%20with%20a%20submission%20ID%20(which%20looks%20like%20h26cq3k\\)%20to%20have%20the%20bot%20scan%20your%20submission%20and%20correct%20your%20first%20seen%20date.)'
preprocessed_text = preprocess_text(input_text)
print(preprocessed_text)

   user report                            total submissions   10   first seen in wsb   2 years ago   total comments   332   previous best dd      account age   3 years   scan  comment     submission   


In [19]:

qna_df['question_tokens'] = qna_df['questions'].apply(lambda x: finbert_tokenizer.tokenize(x))
qna_df['answer_tokens'] = qna_df['answers'].apply(lambda x: finbert_tokenizer.tokenize(x))

qna_df['question_tokens'] = qna_df['question_tokens'].apply(lambda x: ['[CLS]'] + x + ['[SEP]'])
qna_df['answer_tokens'] = qna_df['answer_tokens'].apply(lambda x: ['[CLS]'] + x + ['[SEP]'])

qna_df['question_ids'] = qna_df['question_tokens'].apply(lambda x: finbert_tokenizer.convert_tokens_to_ids(x))
qna_df['answer_ids'] = qna_df['answer_tokens'].apply(lambda x: finbert_tokenizer.convert_tokens_to_ids(x))


In [20]:
qna_df

,questions,answers,tags,question_tokens,answer_tokens,question_ids,answer_ids
0,Hi,Hello,welcome,"[[CLS], hi, [SEP]]","[[CLS], hello, [SEP]]","[3, 4325, 4]","[3, 23738, 4]"
1,Hi,Glad to see you again,welcome,"[[CLS], hi, [SEP]]","[[CLS], glad, to, see, you, again, [SEP]]","[3, 4325, 4]","[3, 10079, 9, 153, 40, 1117, 4]"
2,Hi,"Hi there, How can I help you",welcome,"[[CLS], hi, [SEP]]","[[CLS], hi, there, ,, how, can, i, help, you, ...","[3, 4325, 4]","[3, 4325, 112, 585, 283, 110, 44, 904, 40, 4]"
3,Hi,Hello! what you are looking for?,welcome,"[[CLS], hi, [SEP]]","[[CLS], hello, !, what, you, are, looking, for...","[3, 4325, 4]","[3, 23738, 17293, 163, 40, 21, 601, 14, 4642, 4]"
4,Hi,"Hello, how may I help you",welcome,"[[CLS], hi, [SEP]]","[[CLS], hello, ,, how, may, i, help, you, [SEP]]","[3, 4325, 4]","[3, 23738, 585, 283, 32, 44, 904, 40, 4]"
...,...,...,...,...,...,...,...
700,Thank you,You are welcome,Thank you,"[[CLS], thank, you, [SEP]]","[[CLS], you, are, welcome, [SEP]]","[3, 332, 40, 4]","[3, 40, 21, 2133, 4]"
701,Thank you,No problem. Its my pleasure to help you,Thank you,"[[CLS], thank, you, [SEP]]","[[CLS], no, problem, ., its, my, pleasure, to,...","[3, 332, 40, 4]","[3, 141, 3221, 48, 38, 657, 11109, 9, 904, 40, 4]"
702,Thank you so much for your help,Its my pleasure.,Thank you,"[[CLS], thank, you, so, much, for, your, help,...","[[CLS], its, my, pleasure, ., [SEP]]","[3, 332, 40, 96, 406, 14, 185, 904, 4]","[3, 38, 657, 11109, 48, 4]"
703,Thank you so much for your help,You are welcome,Thank you,"[[CLS], thank, you, so, much, for, your, help,...","[[CLS], you, are, welcome, [SEP]]","[3, 332, 40, 96, 406, 14, 185, 904, 4]","[3, 40, 21, 2133, 4]"
